In [2]:
import pandas as pd
import sqlite3 as lite

In [3]:
conn = lite.connect("store1.db")

In [ ]:
# Load inventory data
sales_transactions = pd.read_sql("select * from sales_transactions", conn)
product_dimension = pd.read_sql("select * from ProductDimension", conn)

,date,customer_number,sku,salesPrice,items_left,cases_ordered
0,2024-01-01,1,42360001,2.08,191,16
1,2024-01-01,1,42907001,2.75,83,7
2,2024-01-01,1,43896001,1.03,83,7
3,2024-01-01,1,42367001,3.25,83,7
4,2024-01-01,1,42113001,1.88,83,7


In [21]:
sales_transactions.head()

,date,customer_number,sku,salesPrice,items_left,cases_ordered
0,2024-01-01,1,42360001,2.08,191,16
1,2024-01-01,1,42907001,2.75,83,7
2,2024-01-01,1,43896001,1.03,83,7
3,2024-01-01,1,42367001,3.25,83,7
4,2024-01-01,1,42113001,1.88,83,7


In [19]:
inventory[inventory['Quarter'] == 1].shape

(16160, 12)

In [5]:
product_dimension.head()

,ProductKey,sku,product_name,product_class_id,subcategory,category,department,product_family,size,brandName,supplier
0,1,42081001,Jambalaya Rice Mix,57,Rice,Starchy Foods,Starchy Foods,Food,12 oz,Zatarains,Rowan Warehouse
1,2,42082001,Jambalaya Rice Mix,57,Rice,Starchy Foods,Starchy Foods,Food,8 oz,Zatarains,Rowan Warehouse
2,3,42083001,Guacamole Regular,99,Fresh Fruit,Fruit,Produce,Food,8 oz,Yucatan,Rowan Warehouse
3,4,42084001,Coffee Original Blend,90,Coffee,Hot Beverages,Beverages,Drink,12 oz,Yuban,Rowan Warehouse
4,5,42085001,GoGurt Variety Pack,6,Yogurt,Dairy,Dairy,Food,8 ct,Yoplait,Rowan Warehouse


In [6]:
# Filter for Q1 2024 and Q1 2025
q1_2024 = inventory[inventory['Quarter and Year'] == "Q1 2024"]
q1_2025 = inventory[inventory['Quarter and Year'] == "Q1 2025"]

In [7]:
# Get product details from dimension table
product_details = product_dimension[['ProductKey', 'sku',
                               'product_name', 'category', 'subcategory']]

In [ ]:
def prepare_quarterly_report(df, year_quarter):
    # Aggregate sales by product
    product_sales = df.groupby('ProductKey')[
        'TotalSoldByStoreThisQuarter', "StoreKey"].sum().reset_index()
    

    product_sales = product_sales.rename(
        columns={'TotalSoldByStoreThisQuarter': f'{year_quarter} Sales'})

    # Add rank
    product_sales[f'{year_quarter} Rank'] = product_sales[f'{year_quarter} Sales'].rank(
        ascending=False, method='min').astype(int)

    # Merge with product details
    report = pd.merge(product_sales, product_details,
                      on='ProductKey', how='left')

    return report.sort_values(f'{year_quarter} Rank')

In [18]:
# Prepare reports for both quarters
report_2024q1 = prepare_quarterly_report(q1_2024, '2024Q1')
report_2025q1 = prepare_quarterly_report(q1_2025, '2025Q1')

# Merge the two reports
final_report = pd.merge(
    report_2024q1,
    report_2025q1,
    on=['ProductKey', 'sku', 'product_name', 'category', 'subcategory'],
    how='outer'
)

# Select and reorder columns to match your desired format
final_report = final_report[[
    'sku', 'product_name', 'category', 'subcategory',
    '2024Q1 Rank', '2024Q1 Sales', '2025Q1 Rank', '2025Q1 Sales'
]]

# Sort by 2024Q1 rank to match your example
final_report = final_report.sort_values('2024Q1 Rank')

# Handle NaN values and format sales numbers with commas (optional)
final_report['2024Q1 Sales'] = final_report['2024Q1 Sales'].fillna(0).apply(
    lambda x: f"{int(x):,}")
final_report['2025Q1 Sales'] = final_report['2025Q1 Sales'].fillna(0).apply(
    lambda x: f"{int(x):,}")


final_report.iloc[:40]

# Save to CSV
# final_report.to_csv('product_rankings_comparison.csv', index=False)


,sku,product_name,category,subcategory,2024Q1 Rank,2024Q1 Sales,2025Q1 Rank,2025Q1 Sales
278,42359001,Whole Milk Milk,Dairy,Milk,1.0,"256,792,800",5.0,"2,213,600,004"
277,42358001,2.00% Milk,Dairy,Milk,2.0,"256,177,944",2.0,"2,244,906,708"
276,42357001,2.00% Milk,Dairy,Milk,3.0,"256,110,936",6.0,"2,209,889,088"
274,42355001,1.00% Milk,Dairy,Milk,4.0,"255,870,636",1.0,"2,254,055,112"
279,42360001,Whole Milk Milk,Dairy,Milk,5.0,"253,960,140",3.0,"2,227,959,648"
275,42356001,1.00% Milk,Dairy,Milk,6.0,"253,212,312",4.0,"2,220,856,344"
233,42314001,Squeeze Jelly Grape,Jams and Jellies,Jelly,7.0,"20,743,572",10.0,"171,619,032"
231,42312001,Jam Strawberry,Jams and Jellies,Jam,8.0,"20,610,036",8.0,"175,396,452"
230,42311001,Jam Grape,Jams and Jellies,Jam,9.0,"20,446,452",7.0,"176,205,600"
232,42313001,Jelly Grape,Jams and Jellies,Jam,10.0,"20,391,312",9.0,"174,394,680"
